In [1]:
# pyton 3.7

In [1]:
import requests
import pdftotext
import time
import io
import json
import pandas as pd
from bs4 import BeautifulSoup

### Typical Format

- Call Category
- Location
- Date Report
- Case #
- Date Occurred
- Time Occurred
- Summary
- Dispotion
- Arrest Info (if applicable) 

In [52]:
crime_log_url_addr   = "http://www.police.ucsd.edu/docs/reports/CallsandArrests/Calls_and_Arrests.asp"
crime_log_prefix = "http://www.police.ucsd.edu/docs/reports/CallsandArrests/"

data_fields = ["CALL_CATEGORY", "LOCATION", "DATE_REPORTED", "CASE_NUMBER",
               "DATE_OCCURRED", "TIME_OCCURRED", "SUMMARY", "DISPOSITION", "ARREST", "IS_UPDATE"]

In [53]:
# splits the texts into sublists of their entries
def get_entries(page_text):
    entries_list = []
    entry        = []
    entry_delim  = [(i - 2) for i in range(0, len(page_text)) if page_text[i].startswith("Date Reported")]
    entry_delim.append(len(page_text))
    entries_list = [page_text[ele0:ele1] 
                        for (ele0, ele1) in zip(entry_delim[:-1], entry_delim[1:])] 
    return entries_list

In [54]:
def parse_log_entry(entry_text_list, is_update):
    VAR_LEN_FIELD_PREFIXES = {"Summary": "SUMMARY", "Disposition": "DISPOSITION", "Arrest Date": "ARREST"}
    PREFIX_LIST            = list(VAR_LEN_FIELD_PREFIXES.keys()) + ["Date Reported", "Incident/Case#", 
                                                                    "Date Occurred", "Time Occurred "]
    LAST_FIXED_LEN_FIELD   = 6
    line_i = 0
    prefix = ""
    entry = dict()
    # get all entries that only take up 1 line
    for field in data_fields[0:LAST_FIXED_LEN_FIELD]:
        value = entry_text_list[line_i]
        prefix_filter = list(filter(entry_text_list[line_i].startswith, PREFIX_LIST))
        if(prefix_filter != []):
            value = value.strip(prefix_filter[0]).strip()
        entry[field] = value
        line_i+=1
    # get all entries that can take up multiple lines
    for line_i in range(line_i, len(entry_text_list)):
        prefix_filter = list(filter(entry_text_list[line_i].startswith, PREFIX_LIST))
        if(prefix_filter != []):
            prefix = prefix_filter[0]
        if(VAR_LEN_FIELD_PREFIXES[prefix] in entry):
            entry[VAR_LEN_FIELD_PREFIXES[prefix]] = entry[VAR_LEN_FIELD_PREFIXES[prefix]] + (entry_text_list[line_i].strip())
        else:
            entry[VAR_LEN_FIELD_PREFIXES[prefix]] = entry_text_list[line_i].lstrip(prefix).lstrip(':').lstrip()
    if("ARREST" in entry):
        entry["ARREST"] = True
    else:
        entry["ARREST"] = False
    entry["IS_UPDATE"] = is_update
    return entry

In [55]:
def parse_log_pdf(pdf):    
    DATE_LINE = 2 # index with the dateline in the header
    # iterate through the pages
    corpus = []
    for page in pdf:
        # split page into array of strings based on new line character
        page_text = page.split('\n')
        if ("UPDATE" in page_text[DATE_LINE]): # TODO: move this check earlier in the processing
            is_update = True
        else:
            is_update = False
        entries = get_entries(page_text)
        corpus = corpus + [parse_log_entry(entry, is_update) for entry in entries]
    return corpus

In [56]:
def parse_daily_log(url_day_suffix):
    log_response = requests.get(crime_log_prefix + url_day_suffix)
    
    raw_pdf_data = log_response.content
    
    
    #with open(("./" + url_suffix), 'wb+') as f:
    #    f.write(raw_pdf_data)
    
    with io.BytesIO(raw_pdf_data) as open_pdf_file:
        read_pdf = pdftotext.PDF(open_pdf_file)
        entries = parse_log_pdf(read_pdf)
    time.sleep(0.5)
    return entries

In [57]:
def get_live_log_dates():
    page = requests.get(crime_log_url_addr)
    page_text = BeautifulSoup(page.content)
    option_list = page_text.find_all("option")
    value_list = []
    for option in option_list:
        value_list.append(option["value"])
    dates = value_list
    return dates

In [75]:
def get_logs():
    dates = get_live_log_dates()
    full_data = []
    [full_data.extend(parse_daily_log(date)) for date in dates[1:]]
    return full_data

In [76]:
full_data = get_logs()

In [77]:
len(full_data)

1640

In [80]:
with open("initial_pull.json", "w+") as file:
    json.dump(full_data, file)

## Review Output

In [4]:
with open("./initial_pull.json", "r") as file:
    full_data = json.load(file)

In [5]:
pd.DataFrame(full_data).to_sql("test", con="sqlite:////tmp/test.db", index=False)# .to_excel("initial_pull.xlsx", index=False)

In [6]:
pd.read_excel("./initial_pull.xlsx").to_json("initial_pull.json", orient="records")